In [ ]:
"""
청년 노동시장 정책 스크리닝 모델 - FINAL COMPLETE VERSION

🎯 목적
- 개인의 '노력/선택'이 아닌 구조적 위치(structural position)에 기반하여
  청년의 노동시장 단절 위험과 재연결 가능성을 진단
- 행정 판단 자동화 X (결정권 대체 아님)
- 정책 대상자 1차 선별(Screening) ㅇ
- 정책 자원 배분 효율성(Lift) 검증 ㅇ
- 설명 가능성(Explainability)과 정책 재현성 확보

핵심 철학
- "예측 모델"이 아니라
- "정책 분류 도구 + 위험 환경 진단 도구"

📦 전체 구조
PART 0. 데이터 로드 및 상관관계 분석 (정보누수 제거)
PART 1. 구조적 단절위험 모델 (환경 위험, GBDT)
PART 2. 개인 취업확률 모델 (개인 상태, GBDT + Calibration)
PART 3. 정책 매트릭스 결합 (환경 × 개인)
PART 4. EDA 시각화 (구조 이해)
PART 5. 데이터 기반 페르소나 (정책 언어화)
PART 6. 정책 타겟팅 성능 평가 + 시뮬레이션
"""

# ======================================================
# 0️⃣ 라이브러리 및 환경 설정
# ======================================================

# ------------------------------------------------------
# 파일 및 경로 관리
# ------------------------------------------------------
import glob      # 다수 파일 패턴 로딩 (연도별 CSV)
import os        # 디렉토리 생성 및 경로 관리

# ------------------------------------------------------
# 경고 제거 (모델 로그 가독성 향상)
# ------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

# ------------------------------------------------------
# 수치 계산 및 데이터 처리
# ------------------------------------------------------
import numpy as np
import pandas as pd

# ------------------------------------------------------
# 시각화
# ------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# ------------------------------------------------------
# 머신러닝
# ------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import (
    roc_auc_score,      # 예측력 (순위 기반)
    recall_score,       # 정책 포착률
    precision_score     # 오탐 방지
)

# ------------------------------------------------------
# 모델 저장
# ------------------------------------------------------
import joblib

# ------------------------------------------------------
# 한글 시각화 설정
# ------------------------------------------------------
plt.rcParams["font.family"] = "Gulim"
plt.rcParams["axes.unicode_minus"] = False


# ======================================================
# 📁 결과 저장 경로
# ======================================================

"""
모델 결과는 세 영역으로 분리 저장
- models   : 학습된 모델 객체 (재현성)
- results  : 시각화 결과 (보고서)
- data     : 정책 매트릭스 최종 결과
"""

MODEL_JSON_DIR = "../../results/oxju"        # 시각화 결과
MODEL_DIR      = "../../models/oxju"         # 모델 객체
CSV_DIR        = "../../data/derived/oxju"   # 최종 정책 데이터

# 경로가 없으면 자동 생성
os.makedirs(MODEL_JSON_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(CSV_DIR, exist_ok=True)


# ======================================================
# 🔧 파생변수 생성 함수
# ======================================================
def add_derived_features(df):
    """
    목적
    - 비선형성 보완
    - 정책 해석이 가능한 구조적 파생변수 생성
    - 단순 예측력이 아닌 '설명 가능한 모델' 확보

    설계 원칙
    - 개인 선택/행동 변수 X
    - 구조적 위치·속성 변수 ㅇ
    """

    df = df.copy()

    # -------------------------------
    # 연령대 더미 (비선형 구조 대응)
    # -------------------------------
    df["연령_20대초반"] = (df["만연령"] <= 24).astype(int)
    df["연령_20대후반"] = ((df["만연령"] >= 25) & (df["만연령"] <= 29)).astype(int)
    df["연령_30대초반"] = ((df["만연령"] >= 30) & (df["만연령"] <= 34)).astype(int)
    df["연령_30대후반"] = (df["만연령"] >= 35).astype(int)

    # -------------------------------
    # 성별 × 혼인 상태 (구조적 위치)
    # -------------------------------
    df["여성기혼"] = ((df["성별코드"] == 2) & (df["혼인상태코드"] == 2)).astype(int)
    df["남성미혼"] = ((df["성별코드"] == 1) & (df["혼인상태코드"] == 1)).astype(int)
    df["여성미혼"] = ((df["성별코드"] == 2) & (df["혼인상태코드"] == 1)).astype(int)

    # -------------------------------
    # 학력 구조
    # -------------------------------
    df["고졸이하"] = (df["교육정도_학력구분코드"] <= 3).astype(int)
    df["전문대졸"] = (df["교육정도_학력구분코드"] == 4).astype(int)
    df["대졸이상"] = (df["교육정도_학력구분코드"] >= 5).astype(int)

    # -------------------------------
    # 전공 계열 구조
    # -------------------------------
    df["이공계"] = df["교육정도_계열코드"].isin([3, 50, 60, 70, 80]).astype(int)
    df["인문사회"] = df["교육정도_계열코드"].isin([1, 22, 30, 40]).astype(int)

    # -------------------------------
    # 연령 비선형성
    # -------------------------------
    df["만연령_제곱"] = df["만연령"] ** 2

    return df


# ======================================================
# PART 0 : 청년 미시 데이터 로드
# ======================================================
def load_youth_may_full(path="../../data/youth_may/*.csv", sample_files=3):
    """
    목적
    - 개인 단위 미시 데이터 로드
    - 취업 여부 및 '진성 쉬었음' 상태 정의
    - 취업확률 모델 학습용 데이터 생성

    정책적 의미
    - '노력 부족'이 아닌 상태(state) 기반 분류
    - 진성 쉬었음 = 구조적으로 노동시장과 단절된 상태
    """

    print("\n" + "="*70)
    print("🔬 [청년 미시 데이터 로드]")
    print("="*70)

    # 일부 연도만 샘플 로딩 (개발/검증 목적)
    files = glob.glob(path)[:sample_files]
    dfs = []

    for f in files:
        df_tmp = pd.read_csv(f)
        dfs.append(df_tmp)

    # 다년 데이터 병합
    df_full = pd.concat(dfs, ignore_index=True)

    # 분석 대상 연령 제한
    df_full = df_full[df_full["만연령"].between(20, 39)].copy()

    print(f"√ 로드 완료: {len(df_full):,}명")

    # -------------------------------
    # 취업 여부 (타겟)
    # -------------------------------
    df_full["취업여부"] = (df_full["경제활동상태코드"] == 1).astype(int)

    # -------------------------------
    # 진성 쉬었음 정의
    # -------------------------------
    cond_no_search = df_full["기타활동사항_지난1년내구직활동유무"] == 2
    cond_labor_reason = df_full["기타활동사항_4주내비구직사유코드"].isin([1,2,3,4,5,6])

    df_full["진성쉬었음"] = (cond_no_search & cond_labor_reason).astype(int)

    # -------------------------------
    # 연령대 그룹
    # -------------------------------
    df_full["연령대"] = pd.cut(
        df_full["만연령"],
        [19, 24, 29, 34, 39],
        labels=["20-24", "25-29", "30-34", "35-39"]
    )

    return df_full


In [ ]:
# PART 0 (후반): 상관관계 분석 & 정보누수 제거
def analyze_full_correlation(df, target="진성쉬었음", top_n=30):
    """
    목적
    - 타겟(진성 쉬었음)과 통계적으로 연관된 변수 탐색
    - 사후 정보·행위 변수로 인한 '정보 누수' 사전 차단

    정책적 의미
    - 이 단계는 '변수 선택'이 아니라
      ! 정책적 정당성 검증 단계
    - 이 단계를 거치지 않으면
      → 모델이 예측이 아닌 '판정기'가 됨
    """

    print("\n" + "="*70)
    print(f"🔥 [{target}과 상관관계 분석]")
    print("="*70)

    # ----------------------------------------
    # 1. 수치형 변수만 추출
    # ----------------------------------------
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

    # ----------------------------------------
    # 2. 결측치 과다 변수 제거
    # ----------------------------------------
    valid_cols = [
        col for col in numeric_cols
        if df[col].notna().sum() > len(df) * 0.3
    ]

    print(f"📊 분석 대상 변수 수: {len(valid_cols)}")

    # 상관계수 계산용 데이터
    df_corr = df[valid_cols].dropna()

    # 데이터 부족 시 중단
    if len(df_corr) < 100:
        return None, []

    corr_matrix = df_corr.corr()

    if target not in corr_matrix.columns:
        return corr_matrix, []

    # ----------------------------------------
    # 3. 타겟과의 상관계수 절대값 기준 정렬
    # ----------------------------------------
    target_corr = corr_matrix[target].abs().sort_values(ascending=False)

    # ----------------------------------------
    # 4. 시각화 (필터링 전)
    # ----------------------------------------
    plt.figure(figsize=(12, 8))

    top_vars = target_corr.drop(target).head(top_n)

    # 상관 0.1 이상 강조
    colors = ['#ff6b6b' if x > 0.1 else '#95a5a6' for x in top_vars.values]

    plt.barh(range(len(top_vars)), top_vars.values, color=colors)
    plt.yticks(range(len(top_vars)), top_vars.index, fontsize=9)
    plt.xlabel("상관계수 (절대값)")
    plt.title(f"'{target}'과 상관관계 TOP {top_n} (필터링 전)")
    plt.axvline(x=0.1, color='red', linestyle='--', alpha=0.5)

    plt.tight_layout()
    plt.savefig(f"{MODEL_JSON_DIR}/correlation_bar_before_filter.png", dpi=150)
    plt.show()

    # ----------------------------------------
    # 5. 정보누수 블랙리스트
    # ----------------------------------------
    """
    X 제거 대상
    - 구직 여부
    - 현재 상태
    - 사후 정보
    - 정책 개입 이후 알 수 있는 정보
    """

    BLACKLIST = [
        "진성쉬었음", "취업여부",
        "기타활동사항_지난1년내구직활동유무",
        "기타활동사항_4주내비구직사유코드",
        "경제활동상태", "주요활동상태",
        "미취업", "전직", "퇴직", "이직", "이전직장",
        "현재일관련", "일시휴직", "근로시간", 
        "부업", "무급가족", "종사상지위",
        "구직사항", "구직여부",
        "취업희망", "취업가능성",
        "조사연월", "가중치",
    ]

    high_corr_vars = []
    excluded_count = 0

    # ----------------------------------------
    # 6. 안전 변수 선별
    # ----------------------------------------
    for var in target_corr.index:
        if var == target:
            continue

        # 블랙리스트 필터
        if any(keyword in var for keyword in BLACKLIST):
            excluded_count += 1
            continue

        # 결측률 과다 제거
        if df[var].isna().sum() / len(df) > 0.7:
            excluded_count += 1
            continue

        # 최소 상관 기준
        if target_corr[var] >= 0.02:
            high_corr_vars.append(var)

    print("\n√ 필터링 결과")
    print(f"   - 제외 변수: {excluded_count}")
    print(f"   - 사용 가능 변수: {len(high_corr_vars)}")

    return corr_matrix, high_corr_vars


In [ ]:
# PART 1: 구조적 단절위험 모델 (환경 위험)
def load_eaps_year(path="../../data/eaps_year/*.csv"):
    """
    목적
    - 개인이 아닌 '노동시장 환경 위험' 생성
    - 지역·연령·성별 구조적 단절 확률 추정

    핵심 개념
    - 개인 책임 X
    - 환경 리스크 ㅇ
    """

    USE_COLS = [
        "조사연월", "만연령", "성별코드",
        "교육정도_학력구분코드", "교육정도_계열코드",
        "혼인상태코드", "경제활동상태코드",
        "비경제활동구직의사여부"
    ]

    dfs = [pd.read_csv(f, usecols=USE_COLS) for f in glob.glob(path)]
    df = pd.concat(dfs, ignore_index=True)

    df = df[df["만연령"].between(20, 39)]

    # 연령대 생성
    df["연령대"] = pd.cut(
        df["만연령"], [19, 24, 29, 34, 39],
        labels=["20-24", "25-29", "30-34", "35-39"]
    )

    # ----------------------------------------
    # 구조적 단절 정의
    # ----------------------------------------
    df["시장단절"] = (
        (df["경제활동상태코드"] == 3) &
        (df["비경제활동구직의사여부"] == 2)
    ).astype(int)

    return df


In [ ]:
def run_disconnect_model_enhanced(df):
    """
    GBDT 기반 구조적 단절 위험 모델

    이유
    - 비선형 구조 대응
    - 변수 상호작용 자동 포착
    """

    df = add_derived_features(df)

    FEATURES = [
        "성별코드", "만연령", "만연령_제곱",
        "교육정도_학력구분코드", "교육정도_계열코드",
        "혼인상태코드",
        "여성기혼", "남성미혼", "여성미혼",
        "고졸이하", "대졸이상",
        "연령_20대초반", "연령_20대후반",
        "연령_30대초반", "연령_30대후반"
    ]

    # 모델 입력 구성
    dfm = df[FEATURES + ["시장단절"]].dropna()

    X = pd.get_dummies(dfm[FEATURES], drop_first=True)
    y = dfm["시장단절"]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42
    )

    # ----------------------------------------
    # GBDT 모델
    # ----------------------------------------
    model = GradientBoostingClassifier(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=3,
        min_samples_leaf=50,
        random_state=42
    )

    model.fit(X_train, y_train)

    y_prob = model.predict_proba(X_test)[:, 1]

    print("\n[구조적 단절 위험 모델]")
    print(f"ROC-AUC: {roc_auc_score(y_test, y_prob):.4f}")

    # 전체 데이터에 위험 확률 부여
    X_all = pd.get_dummies(df[FEATURES], drop_first=True)
    X_all = X_all.reindex(columns=X.columns, fill_value=0)

    df["단절위험확률"] = model.predict_proba(X_all)[:, 1]

    return (
        model,
        X_train, y_train,
        X_test, y_test,
        df
    )


In [ ]:
# PART 2: 개인 취업확률 모델 (Calibration 포함)
def train_employment_gbdt_enhanced(df, additional_features=[]):
    """
    목적
    - 개인의 구조적 위치 기반 취업 가능성 추정
    - 확률의 '해석 가능성' 확보 (Calibration)

    핵심
    - 노력/의지 변수 X
    - 구조적 상태 변수 ㅇ
    """

    df = add_derived_features(df)

    BASE_FEATURES = [
        "성별코드", "만연령", "만연령_제곱",
        "교육정도_학력구분코드", "교육정도_계열코드",
        "혼인상태코드",
        "연령_20대초반", "연령_20대후반",
        "연령_30대초반", "연령_30대후반",
        "여성기혼", "남성미혼", "여성미혼",
        "고졸이하", "전문대졸", "대졸이상",
        "이공계", "인문사회"
    ]

    # ----------------------------------------
    # 상관분석에서 추출된 안전 변수 추가
    # ----------------------------------------
    NEW_FEATURES = []
    for f in additional_features[:15]:
        if f in BASE_FEATURES:
            continue
        if f not in df.columns:
            continue
        if not pd.api.types.is_numeric_dtype(df[f]):
            continue
        if df[f].isna().sum() / len(df) > 0.7:
            continue
        NEW_FEATURES.append(f)

    FEATURES = BASE_FEATURES + NEW_FEATURES

    print(f"\n√ 취업 모델 변수 수: {len(FEATURES)}")
    
    # ----------------------------------------
    # 학습 데이터 구성
    # ----------------------------------------
    
    print(f"   - 기본: {len(BASE_FEATURES)}개, 추가: {len(NEW_FEATURES)}개")
    if NEW_FEATURES:
        print(f"   - 추가된 변수:")
        for i, v in enumerate(NEW_FEATURES, 1):
            print(f"      {i}. {v}")
    
    keep_cols = [
        "조사연월", "연령대", "진성쉬었음", 
        "성별코드", "만연령", "교육정도_학력구분코드", "취업여부"
    ]
    
    all_cols = list(dict.fromkeys(FEATURES + keep_cols))
    
    dfm = df[all_cols].dropna()
    
    train_df, test_df = train_test_split(
        dfm, test_size=0.2, stratify=dfm["취업여부"], random_state=42
    )
    
    X_train = pd.get_dummies(train_df[FEATURES], drop_first=True)
    X_test = pd.get_dummies(test_df[FEATURES], drop_first=True)
    X_test = X_test.reindex(columns=X_train.columns, fill_value=0)
    
    y_train = train_df["취업여부"]
    y_test = test_df["취업여부"]

    # ----------------------------------------
    # GBDT + Isotonic Calibration
    # ----------------------------------------
    base = GradientBoostingClassifier(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=3,
        min_samples_leaf=30,
        random_state=42
    )

    model = CalibratedClassifierCV(base, method="isotonic", cv=3)
    model.fit(X_train, y_train)

    y_prob = model.predict_proba(X_test)[:, 1]

    print("\n[취업 확률 모델]")
    print(f"AUC       : {roc_auc_score(y_test, y_prob):.4f}")
    print(f"Recall    : {recall_score(y_test, y_prob >= 0.5):.4f}")
    print(f"Precision : {precision_score(y_test, y_prob >= 0.5):.4f}")

    test_df = test_df.copy()
    test_df["취업예측확률"] = y_prob

    return (
        model,
        X_train, y_train,
        X_test, y_test,
        test_df
    )


In [ ]:
# PART 3: 정책 매트릭스 결합 (환경 × 개인)
def build_hybrid_policy_matrix(df_main, df_test):
    """
    목적
    - PART 1(환경 위험) + PART 2(개인 취업확률)을 결합하여
      정책 대상자 스크리닝을 위한 '정책 매트릭스'를 생성

    핵심 아이디어 (이 모델의 정체성)
    - 개인 단위 확률만 보면 '개인 책임'으로 해석될 위험이 있음
    - 환경 단위 위험만 보면 '개인 다양성'을 놓침

    따라서
    - 환경 위험(구조) × 개인 취업확률(개인)을 결합한 2차원 분류가 핵심

    결과적으로 정책군은 4개로 나뉜다:
    1) 고착형 단절 위험군       : 환경 위험 ↑, 개인 취업확률 ↓
    2) 재연결 가능 단절군       : 환경 위험 ↑, 개인 취업확률 ↑
    3) 취업취약 비단절군        : 환경 위험 ↓, 개인 취업확률 ↓
    4) 안정군                  : 환경 위험 ↓, 개인 취업확률 ↑

    출력
    - df_policy : 개인 단위 정책유형 + 재연결지수 포함
    - risk_env  : (조사연월×성별×연령대) 환경 위험 테이블
    """

    # ----------------------------------------
    # 1. 환경 위험을 만들기 위한 KEY 정의
    # ----------------------------------------
    # 동일한 구조적 환경을 공유하는 집단 단위
    # (연월, 성별, 연령대) 기준으로 그룹핑
    KEYS = ["조사연월", "성별코드", "연령대"]

    # 혼합형 merge를 위해 타입 통일
    df_main["조사연월"] = df_main["조사연월"].astype(str)
    df_test["조사연월"] = df_test["조사연월"].astype(str)

    # ----------------------------------------
    # 2. 환경 위험(단절위험_환경평균) 계산
    # ----------------------------------------
    # PART 1의 개인별 단절위험확률을
    # 환경 단위(연월×성별×연령대) 평균으로 집계
    risk_env = (
        df_main
        .groupby(KEYS)
        .agg(단절위험_환경평균=("단절위험확률", "mean"))
        .reset_index()
    )

    risk_env["조사연월"] = risk_env["조사연월"].astype(str)

    # ----------------------------------------
    # 3. 개인 데이터(df_test)에 환경 위험 merge
    # ----------------------------------------
    df_policy = df_test.merge(risk_env, on=KEYS, how="left")

    # 만약 매칭 실패(해당 환경 조합이 없을 때) -> 전체 평균으로 대체
    # 정책 적용시 "환경정보가 없는 개인"을 버리면 커버리지 손상
    df_policy["단절위험_환경평균"].fillna(
        df_main["단절위험확률"].mean(), inplace=True
    )

    # ----------------------------------------
    # 4. 연령대별 컷오프(quantile) 설정
    # ----------------------------------------
    """
    왜 연령대별로 컷오프를 따로 잡나?
    - 20-24, 35-39의 기본 취업 확률 분포 자체가 다름
    - 하나의 공통 임계값(예: 0.5)을 쓰면 연령 편향이 생김
    - 따라서 연령대 내부에서 상대적 위치로 분류

    RISK_Q: 환경 위험의 상위 몇 %를 "위험군"으로 볼지
    JOB_Q : 취업확률의 하위 몇 %를 "취업취약"으로 볼지
    """
    # 🔥 집중 타겟팅용 (Lift 상승)
    RISK_Q = {"20-24": 0.75, "25-29": 0.75, "30-34": 0.80, "35-39": 0.80}
    JOB_Q  = {"20-24": 0.30, "25-29": 0.30, "30-34": 0.35, "35-39": 0.35}

    # 정책유형 컬럼 초기화
    df_policy["정책유형"] = None

    # ----------------------------------------
    # 5. 연령대별로 위험/취업 컷을 구해 4분면 분류
    # ----------------------------------------
    for age in ["20-24", "25-29", "30-34", "35-39"]:
        age_mask = df_policy["연령대"] == age
        age_data = df_policy[age_mask]

        # 샘플이 너무 적으면 분위수 컷 자체가 불안정 -> 스킵
        if len(age_data) < 30:
            continue

        # 환경 위험 컷: 상위 risk_q 이상이면 "위험 환경"
        risk_cut = age_data["단절위험_환경평균"].quantile(RISK_Q[age])

        # 취업 확률 컷: 하위 job_q 미만이면 "취업 취약"
        job_cut  = age_data["취업예측확률"].quantile(JOB_Q[age])

        # np.select로 4분면 분류
        df_policy.loc[age_mask, "정책유형"] = np.select(
            [
                # 1) 위험 환경 & 취업 취약
                (age_data["단절위험_환경평균"] >= risk_cut) &
                (age_data["취업예측확률"] < job_cut),

                # 2) 위험 환경 & 취업 가능
                (age_data["단절위험_환경평균"] >= risk_cut) &
                (age_data["취업예측확률"] >= job_cut),

                # 3) 안정 환경 & 취업 취약
                (age_data["단절위험_환경평균"] < risk_cut) &
                (age_data["취업예측확률"] < job_cut),
            ],
            ["고착형 단절 위험군", "재연결 가능 단절군", "취업취약 비단절군"],
            default="안정군"
        )

    # ----------------------------------------
    # 6. 재연결지수 정의 (단일 스코어)
    # ----------------------------------------
    """
    재연결지수(Reconnection Index)
    - 정책 집행 실무에서 2차원 매트릭스는 직관적이지만
      '우선순위'를 정할 땐 단일 점수가 더 필요함

    설계 철학
    - 재연결 가능성 = (환경의 안전함) + (개인의 취업 가능성)
    - 단절위험은 낮을수록 좋으므로 (1 - 위험)
    - 현재 가중치는 0.3 / 0.7 로 개인취업확률을 더 크게 반영
      → "환경이 나쁘더라도 개인 회복 가능성이 있으면 지원 효율이 높다"
      라는 정책적 가정에 기반

    ※ 가중치 자체는 정책목적에 따라 바뀔 수 있음
    """
    df_policy["재연결지수"] = (
        0.3 * (1 - df_policy["단절위험_환경평균"]) +
        0.7 * df_policy["취업예측확률"]
    ).round(3)
    
    # ----------------------------------------
    # 7. 집중 타겟팅용 지원 필요도 점수
    # ----------------------------------------
    """
    지원필요도점수 (Need Score)
    - 위험이 높고
    - 취업확률이 낮을수록
    - 점수가 높아짐
    """
    df_policy["지원필요도점수"] = (
        0.7 * df_policy["단절위험_환경평균"] +
        0.3 * (1 - df_policy["취업예측확률"])
    ).round(4)


    return df_policy, risk_env


In [ ]:
# PART 4: EDA 시각화 (정책군의 구조 확인)
def plot_eda_visualizations(df_policy):
    """
    목적
    - 정책 유형별 분포/구조를 직관적으로 확인
    - 정책 대상자의 “어떤 특성의 집단이 어디에 몰리는지” 설명자료 생성

    생성 그래프
    1) Ridge Plot        : 정책유형별 연령 분포 차이
    2) Scatter Plot      : (취업확률, 환경위험) 2차원 정책 분포
    3) Boxplot           : 정책유형별 재연결지수 분포
    4) Targeting Barplot : 정책유형별 진성쉬었음 비율(Lift 직관)
    """

    print("\n" + "="*70)
    print("※ [EDA 시각화 생성]")
    print("="*70)

    # --------------------------------------------------
    # 1) Ridge Plot (여기서는 KDE 곡선을 겹쳐 표현)
    # --------------------------------------------------
    fig, ax = plt.subplots(figsize=(12, 6))

    # 정책유형별로 KDE를 겹쳐 그려 분포 비교
    for group in df_policy["정책유형"].unique():
        subset = df_policy[df_policy["정책유형"] == group]

        # 표본이 적으면 KDE가 불안정 -> 최소 개수 제한
        if len(subset) > 10:
            subset["만연령"].plot(kind='kde', ax=ax, label=group, alpha=0.6)

    ax.set_xlabel("만연령")
    ax.set_title("정책 유형별 연령 분포")
    ax.legend()

    plt.tight_layout()
    plt.savefig(f"{MODEL_JSON_DIR}/ridge_plot_final.png", dpi=150, bbox_inches='tight')
    plt.show()

    # --------------------------------------------------
    # 2) Cluster Scatter (정책유형이 실제로 분리되는지 확인)
    # --------------------------------------------------
    fig, ax = plt.subplots(figsize=(10, 8))

    for group in df_policy["정책유형"].unique():
        subset = df_policy[df_policy["정책유형"] == group]
        ax.scatter(
            subset["취업예측확률"],
            subset["단절위험_환경평균"],
            label=group,
            alpha=0.6,
            s=30
        )

    ax.set_xlabel("취업예측확률")
    ax.set_ylabel("단절위험(환경평균)")
    ax.set_title("정책 클러스터 분포 (확률×환경)")
    ax.legend()
    ax.grid(alpha=0.3)

    plt.tight_layout()
    plt.savefig(f"{MODEL_JSON_DIR}/cluster_scatter.png", dpi=150, bbox_inches='tight')
    plt.show()

    # --------------------------------------------------
    # 3) Boxplot (정책유형별 재연결지수 분포)
    # --------------------------------------------------
    fig, ax = plt.subplots(figsize=(10, 6))

    # pandas 내장 boxplot 사용
    df_policy.boxplot(column="재연결지수", by="정책유형", ax=ax)

    ax.set_title("정책 유형별 재연결 지수 분포")
    ax.set_xlabel("정책유형")
    ax.set_ylabel("재연결지수")

    # pandas boxplot은 기본적으로 suptitle이 들어가므로 제거
    plt.suptitle("")

    plt.tight_layout()
    plt.savefig(f"{MODEL_JSON_DIR}/boxplot_reconnect.png", dpi=150, bbox_inches='tight')
    plt.show()

    # --------------------------------------------------
    # 4) Targeting 성능 (정책유형별 진성쉬었음 비율)
    # --------------------------------------------------
    # 정책군별 '진성쉬었음' 평균 = 해당 정책군이 실제 타겟을 얼마나 포함하는지
    segment_stats = df_policy.groupby("정책유형")["진성쉬었음"].agg(["mean", "count"])
    segment_stats["mean"] = segment_stats["mean"] * 100
    segment_stats = segment_stats.sort_values("mean", ascending=False)

    fig, ax = plt.subplots(figsize=(10, 6))

    # 위험도가 10% 이상이면 강조 색상 부여
    colors = ['#e74c3c' if x > 10 else '#95a5a6' for x in segment_stats["mean"]]

    ax.barh(range(len(segment_stats)), segment_stats["mean"], color=colors)
    ax.set_yticks(range(len(segment_stats)))
    ax.set_yticklabels(segment_stats.index)
    ax.set_xlabel("진성 쉬었음 비율 (%)")
    ax.set_title("정책 유형별 타겟팅 성능(진성 쉬었음 포착률)")

    # 전체 평균선 (정책군이 평균보다 얼마나 높은지 직관 제공)
    ax.axvline(
        x=df_policy["진성쉬었음"].mean() * 100,
        color='red', linestyle='--', label='전체 평균'
    )
    ax.legend()

    plt.tight_layout()
    plt.savefig(f"{MODEL_JSON_DIR}/targeting_accuracy.png", dpi=150, bbox_inches='tight')
    plt.show()

    print("시각화 완료")


In [ ]:
# PART 5: 데이터 기반 페르소나 (정책 언어로 변환)
def generate_personas(df_policy):
    """
    목적
    - 정책 매트릭스의 4개 군집을
      '정책 커뮤니케이션 가능한 페르소나'로 요약

    이유
    - 정책은 숫자만으로 설득되지 않음
    - 정책유형별 대표 특성을 요약해야
      실무자/의사결정자가 이해·설명 가능
    """

    print("\n" + "="*70)
    print("🎭 [데이터 기반 페르소나]")
    print("="*70)

    # 정책유형을 고정 순서로 출력 (보고서 정렬용)
    for group in ["고착형 단절 위험군", "재연결 가능 단절군", "취업취약 비단절군", "안정군"]:
        subset = df_policy[df_policy["정책유형"] == group]

        # 해당 그룹이 없으면 스킵
        if len(subset) == 0:
            continue

        # ----------------------------------------
        # 페르소나 핵심 요약 통계
        # ----------------------------------------
        avg_age = subset["만연령"].mean()

        # 최빈값 기반 대표 성별
        gender_mode = subset["성별코드"].mode()[0] if len(subset["성별코드"].mode()) > 0 else 1
        gender_str = "여성" if gender_mode == 2 else "남성"

        # 최빈값 기반 대표 학력
        edu_mode = subset["교육정도_학력구분코드"].mode()[0] if len(subset["교육정도_학력구분코드"].mode()) > 0 else 5
        edu_map = {1: "중졸이하", 2: "고졸", 3: "고졸", 4: "전문대졸", 5: "대졸", 6: "대졸이상"}
        edu_str = edu_map.get(edu_mode, "대졸")

        # 진단 지표들
        emp_rate      = subset["취업여부"].mean() * 100
        risk_avg      = subset["단절위험_환경평균"].mean() * 100
        reconnect_avg = subset["재연결지수"].mean()
        target_rate   = subset["진성쉬었음"].mean() * 100

        # ----------------------------------------
        # 출력 (보고서 복붙용)
        # ----------------------------------------
        print(f"\n{'='*60}")
        print(f"🏷️  [{group}]")
        print(f"{'='*60}")
        print(f"인구 요약: 평균 {avg_age:.1f}세 / {edu_str} {gender_str}")
        print(f"※ 진단 요약: 취업{emp_rate:.1f}% | 환경위험{risk_avg:.1f}% | 재연결{reconnect_avg:.2f}")
        print(f"※  진성 쉬었음 비율: {target_rate:.1f}%")


In [ ]:
# PART 6: 최종 성능 평가 + 시뮬레이션 (Lift, 커버리지)

def evaluate_total_matrix_performance(df):
    """
    목적
    - 취업 모델 성능(AUC) 확인
    - 정책군별 '진성쉬었음' 포착률을 통해 Lift 측정
    - 재연결지수가 실제 취업과 연관되는지(신뢰도) 점검
    - 정책군 커버리지(정책 자원 배분 규모) 확인
    - 간단 예산 가중치 기반 시뮬레이션(효율 비교) 제공
    """

    print("\n" + "="*70)
    print("🚀 [최종 성능 평가]")
    print("="*70)

    # ----------------------------------------
    # 1. 취업확률 모델 AUC
    # ----------------------------------------
    y_true = df["취업여부"]
    auc = roc_auc_score(y_true, df["취업예측확률"])
    print(f"\n1. 취업 모델 AUC: {auc:.4f}")

    # ----------------------------------------
    # 2. 정책유형별 진성쉬었음 평균/인원
    # ----------------------------------------
    segment_stats = df.groupby("정책유형")["진성쉬었음"].agg(["mean", "count"])
    segment_stats["mean"] = segment_stats["mean"] * 100
    segment_stats = segment_stats.sort_values("mean", ascending=False)

    # Lift 계산 대상 그룹 (가장 고위험 타겟)
    target_group = "고착형 단절 위험군"

    captured_rate = segment_stats.loc[target_group, "mean"] if target_group in segment_stats.index else 0
    avg_rate = df["진성쉬었음"].mean() * 100

    # Lift = (타겟군 위험률 / 전체 평균 위험률)
    lift = captured_rate / avg_rate if avg_rate > 0 else 0

    print(f"\n2. 타겟팅 성능")
    print(f"   - 전체 평균: {avg_rate:.2f}%")
    print(f"   - 고착형 단절 위험군: {captured_rate:.2f}%")
    print(f"   Lift: {lift:.2f}배")

    # ----------------------------------------
    # 3. 재연결지수 신뢰도
    # ----------------------------------------
    # 재연결지수가 높을수록 실제 취업이 높아야 정책 점수로 의미가 있음
    corr = df[["재연결지수", "취업여부"]].corr().iloc[0, 1]
    print(f"\n3. 재연결 지수 신뢰도(취업과 상관): {corr:.4f}")

    # ----------------------------------------
    # 4. 커버리지 (정책군별 구성비)
    # ----------------------------------------
    print(f"\n4. 매트릭스 커버리지:")
    coverage = df["정책유형"].value_counts(normalize=True) * 100
    for group, pct in coverage.items():
        print(f"   - {group}: {pct:.1f}%")

    # ----------------------------------------
    # 5. 정책군별 진성쉬었음 비율 상세
    # ----------------------------------------
    print(f"\n5. 정책 유형별 진성쉬었음 비율:")
    for group in segment_stats.index:
        rate = segment_stats.loc[group, "mean"]
        count = segment_stats.loc[group, "count"]
        print(f"   - {group}: {rate:.2f}% ({int(count)}명)")

    # ==================================================
    # 정책 시뮬레이션 그래프 (예산 가중치 기반)
    # ==================================================
    print("\n" + "="*70)
    print("※ [정책 시뮬레이션 결과 시각화]")
    print("="*70)

    # ----------------------------------------
    # 예산 가중치(예시)
    # ----------------------------------------
    """
    의미
    - 실제 예산이 아니라 '상대적 자원 투입 강도'를 가정한 시뮬레이션
    - 고착형은 집중 투입(100), 안정군은 최소(10) 같은 형태
    - 정책 논리 시뮬레이션용
    """
    policy_budget = {
        "고착형 단절 위험군": 100,
        "재연결 가능 단절군": 70,
        "취업취약 비단절군": 50,
        "안정군": 10
    }

    # ----------------------------------------
    # 효과성(effectiveness) 계산
    # ----------------------------------------
    """
    효과성 = (진성쉬었음 비율 × 대상 인원 × 예산가중치)
    - 여기서 진성쉬었음 비율을 '지원 필요도'의 대리변수로 보고
    - 예산가중치로 정책의 “강도”를 반영한 종합 점수
    """
    results = []
    for group in segment_stats.index:
        rate = segment_stats.loc[group, "mean"]
        count = segment_stats.loc[group, "count"]
        budget = policy_budget.get(group, 0)

        effectiveness = (rate / 100) * count * (budget / 100)

        results.append({
            "정책유형": group,
            "위험비율": rate,
            "대상인원": count,
            "예산가중치": budget,
            "효과성": effectiveness
        })

    df_results = pd.DataFrame(results).sort_values("효과성", ascending=False)

    # ----------------------------------------
    # 그래프 1: 2x2 패널 (원본 코드 그대로)
    # ----------------------------------------
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))

    # 1) 위험비율
    ax1 = axes[0, 0]
    colors1 = ['#e74c3c' if x > 10 else '#3498db' if x > 5 else '#95a5a6' for x in df_results["위험비율"]]
    ax1.barh(df_results["정책유형"], df_results["위험비율"], color=colors1)
    ax1.axvline(x=avg_rate, color='red', linestyle='--', linewidth=2, label=f'전체평균 {avg_rate:.1f}%')
    ax1.set_xlabel("진성 쉬었음 비율 (%)")
    ax1.set_title("정책 유형별 위험도")
    ax1.legend()
    ax1.grid(axis='x', alpha=0.3)

    # 2) 대상인원
    ax2 = axes[0, 1]
    colors2 = ['#9b59b6', '#3498db', '#e67e22', '#95a5a6']
    ax2.barh(df_results["정책유형"], df_results["대상인원"], color=colors2)
    ax2.set_xlabel("대상 인원 (명)")
    ax2.set_title("정책 유형별 규모")
    ax2.grid(axis='x', alpha=0.3)

    # 3) 예산가중치
    ax3 = axes[1, 0]
    colors3 = ['#e74c3c', '#f39c12', '#f1c40f', '#95a5a6']
    ax3.barh(df_results["정책유형"], df_results["예산가중치"], color=colors3)
    ax3.set_xlabel("예산 가중치")
    ax3.set_title("정책별 투입 예산 (상대값)")
    ax3.grid(axis='x', alpha=0.3)

    # 4) 종합 효과성
    ax4 = axes[1, 1]
    colors4 = ['#27ae60' if i == 0 else '#95a5a6' for i in range(len(df_results))]
    ax4.barh(df_results["정책유형"], df_results["효과성"], color=colors4)
    ax4.set_xlabel("정책 효과성 (종합 점수)")
    ax4.set_title("정책별 투입 대비 효과")
    ax4.grid(axis='x', alpha=0.3)

    plt.tight_layout()
    plt.savefig(f"{MODEL_JSON_DIR}/policy_simulation_results.png", dpi=150, bbox_inches='tight')
    plt.show()

    # ----------------------------------------
    # 그래프 2: Lift 비교
    # ----------------------------------------
    fig, ax = plt.subplots(figsize=(10, 6))

    lifts = []
    for group in segment_stats.index:
        rate = segment_stats.loc[group, "mean"]
        group_lift = rate / avg_rate if avg_rate > 0 else 0
        lifts.append({"정책유형": group, "Lift": group_lift})

    df_lifts = pd.DataFrame(lifts).sort_values("Lift", ascending=False)

    colors_lift = ['#e74c3c' if x > 2 else '#f39c12' if x > 1 else '#95a5a6' for x in df_lifts["Lift"]]
    ax.barh(df_lifts["정책유형"], df_lifts["Lift"], color=colors_lift)
    ax.axvline(x=1.0, color='black', linestyle='--', linewidth=2, label='기준선 (1.0)')
    ax.axvline(x=2.0, color='red', linestyle='--', linewidth=1.5, alpha=0.5, label='목표 (2.0)')
    ax.set_xlabel("Lift (배수)")
    ax.set_title("정책 유형별 타겟팅 Lift")
    ax.legend()
    ax.grid(axis='x', alpha=0.3)

    # 값 표시(막대 옆에 텍스트)
    for i, (idx, row) in enumerate(df_lifts.iterrows()):
        ax.text(row["Lift"] + 0.05, i, f'{row["Lift"]:.2f}배', va='center', fontsize=10, weight='bold')

    plt.tight_layout()
    plt.savefig(f"{MODEL_JSON_DIR}/policy_lift_comparison.png", dpi=150, bbox_inches='tight')
    plt.show()

    print("정책 시뮬레이션 그래프 생성 완료")

    # ==================================================
    # ※ 아래 블록은 원본 코드에 "동일 평가 출력이 한 번 더" 존재
    # (요청대로 삭제하지 않고 그대로 두되, 의미를 주석으로 설명)
    # ==================================================
    """
    왜 중복 출력이 있나?
    - 개발 중간에 시뮬레이션 추가하면서
      기존 평가 출력 블록을 아래에 다시 붙여 넣은 형태로 보임
    - 기능상 문제는 없지만 출력이 2번 반복됨
    - 나중에 정리할 땐 아래 중복 블록을 제거하면 깔끔해짐
      (하지만 현재는 '삭제 금지' 조건 때문에 유지)
    """

    print("\n" + "="*70)
    print("🚀 [최종 성능 평가]")
    print("="*70)

    y_true = df["취업여부"]
    auc = roc_auc_score(y_true, df["취업예측확률"])
    print(f"\n1. 취업 모델 AUC: {auc:.4f}")

    segment_stats = df.groupby("정책유형")["진성쉬었음"].agg(["mean", "count"])
    segment_stats["mean"] = segment_stats["mean"] * 100
    segment_stats = segment_stats.sort_values("mean", ascending=False)

    target_group = "고착형 단절 위험군"
    captured_rate = segment_stats.loc[target_group, "mean"] if target_group in segment_stats.index else 0
    avg_rate = df["진성쉬었음"].mean() * 100
    lift = captured_rate / avg_rate if avg_rate > 0 else 0

    print(f"\n2. 타겟팅 성능")
    print(f"   - 전체 평균: {avg_rate:.2f}%")
    print(f"   - 고착형 단절 위험군: {captured_rate:.2f}%")
    print(f"   Lift: {lift:.2f}배")

    corr = df[["재연결지수", "취업여부"]].corr().iloc[0, 1]
    print(f"\n3. 재연결 지수 신뢰도: {corr:.4f}")

    print(f"\n4. 매트릭스 커버리지:")
    coverage = df["정책유형"].value_counts(normalize=True) * 100
    for group, pct in coverage.items():
        print(f"   - {group}: {pct:.1f}%")

    print(f"\n5. 정책 유형별 진성쉬었음 비율:")
    for group in segment_stats.index:
        rate = segment_stats.loc[group, "mean"]
        count = segment_stats.loc[group, "count"]
        print(f"   - {group}: {rate:.2f}% ({int(count)}명)")


In [ ]:
# MAIN 실행부: 전체 파이프라인 (오케스트레이션)
print("="*70)
print("🚀 FINAL COMPLETE PIPELINE START")
print("="*70)

# ======================================================
# STEP 0: 청년 미시 데이터 로드 + 상관관계 분석
# ======================================================
"""
여기서 하는 일
1) youth_may(개인 단위) 로드
2) 진성쉬었음 라벨 생성
3) 타겟과 연관된 변수 탐색
4) 정보 누수 필터링 후, 안전한 추가 변수 후보(high_corr_vars) 생성

high_corr_vars는 PART 2 취업 모델에 "추가 변수 후보"로만 사용됨
(즉, 상관분석이 곧 변수 확정이 아니라 '후보 풀' 생성 단계)
"""
df_full = load_youth_may_full()
corr_matrix, high_corr_vars = analyze_full_correlation(df_full)

# ======================================================
# STEP 1: 단절위험 모델 학습 (환경 위험)
# ======================================================
print("\n" + "="*70)
print("STEP 1: 단절위험 모델 학습 (GBDT Enhanced)")
print("="*70)

"""
여기서 하는 일
- eaps_year(연간) 데이터로
  '환경 위험(구조적 단절)' 모델을 학습한다.

결과:
- df_main에 개인별 단절위험확률이 붙고
- 이후 KEY(연월×성별×연령대)로 평균내어 환경위험을 만든다.
"""
df_main = load_eaps_year()
df_main = add_derived_features(df_main)
(
    disconnect_model,
    Xd_train, yd_train,
    Xd_test, yd_test,
    df_main
) = run_disconnect_model_enhanced(df_main)

# ======================================================
# STEP 2: 취업확률 모델 학습 (개인 취업확률)
# ======================================================
print("\n" + "="*70)
print("STEP 2: 취업확률 모델 학습")
print("="*70)

"""
여기서 하는 일
- df_full(개인 단위)로 취업확률을 학습한다.
- Calibration(isotonic) 적용으로 예측확률이 정책적으로 해석 가능해진다.
- additional_features=high_corr_vars로
  '안전 변수 후보' 일부를 추가해 성능/설명력을 보완한다.
"""
(
    employment_model,
    X_train, y_train,
    X_test, y_test,
    df_test
)= train_employment_gbdt_enhanced(
    df_full, additional_features=high_corr_vars
)

# ======================================================
# STEP 3: 정책 매트릭스 구성 (환경×개인 결합)
# ======================================================
print("\n" + "="*70)
print("STEP 3: 정책 매트릭스 구성")
print("="*70)

"""
여기서 하는 일
- df_main에서 환경위험(단절위험_환경평균) 생성
- df_test(개인 취업확률)에 merge
- 연령대별 분위수 컷으로 정책유형을 분류
- 재연결지수까지 계산하여 단일 스코어 제공
"""
df_policy, risk_env = build_hybrid_policy_matrix(df_main, df_test)

print("\n[정책유형 분포]")
print(df_policy["정책유형"].value_counts())

print("\n[연령대별 분포]")
print(pd.crosstab(df_policy["연령대"], df_policy["정책유형"]))

# ======================================================
# STEP 4: EDA 시각화
# ======================================================
"""
정책 보고서용 그림 자동 생성
- ridge_plot_final.png
- cluster_scatter.png
- boxplot_reconnect.png
- targeting_accuracy.png
"""
plot_eda_visualizations(df_policy)

# ======================================================
# STEP 5: 페르소나 생성
# ======================================================
"""
정책유형별 대표 속성 요약
- 평균 연령, 대표 성별/학력
- 취업률, 위험도, 재연결지수, 진성쉬었음 비율
"""
generate_personas(df_policy)

# ======================================================
# STEP 6: 최종 성능 평가 + 시뮬레이션
# ======================================================
"""
정책모형의 목적은 단순 AUC가 아니라
- 타겟팅 Lift
- 커버리지
- 재연결지수의 정책적 신뢰도
를 함께 보는 것
"""
evaluate_total_matrix_performance(df_policy)

# ======================================================
# 결과 저장 (모델/데이터)
# ======================================================
print("\n" + "="*70)
print("💾 모델 및 결과 저장")
print("="*70)

# 구조적 단절 위험 모델 저장
joblib.dump(disconnect_model, f"{MODEL_DIR}/disconnect_gbdt_final.pkl")

# 취업확률 모델 저장 (Calibration 포함 객체)
joblib.dump(employment_model, f"{MODEL_DIR}/employment_gbdt_final.pkl")

# 정책 매트릭스 결과 저장
df_policy.to_csv(f"{CSV_DIR}/policy_matrix_final.csv", index=False, encoding="utf-8-sig")

print(f"저장 완료!")
print(f"   - 모델: {MODEL_DIR}")
print(f"   - 시각화: {MODEL_JSON_DIR}")
print(f"   - 데이터: {CSV_DIR}")

print("\n" + "="*70)
print("🎉 전체 파이프라인 완료!")
print("="*70)


In [ ]:
# ======================================================
# TEST 1: 구조적 단절 위험 모델 (Train vs Test)
# ======================================================

from sklearn.metrics import roc_auc_score

def test_disconnect_overfitting(model, X_train, y_train, X_test, y_test):
    """
    구조적 단절 위험 모델 오버피팅 점검
    """
    # Train
    y_train_prob = model.predict_proba(X_train)[:, 1]
    train_auc = roc_auc_score(y_train, y_train_prob)

    # Test
    y_test_prob = model.predict_proba(X_test)[:, 1]
    test_auc = roc_auc_score(y_test, y_test_prob)

    print("\n" + "="*60)
    print("🧪 [TEST] 구조적 단절 위험 모델")
    print("="*60)
    print(f"Train AUC : {train_auc:.4f}")
    print(f"Test  AUC : {test_auc:.4f}")
    print(f"Gap       : {train_auc - test_auc:.4f}")

    return {
        "train_auc": train_auc,
        "test_auc": test_auc,
        "gap": train_auc - test_auc
    }


In [ ]:
test_disconnect_overfitting(
    disconnect_model,
    Xd_train, yd_train,
    Xd_test, yd_test
)


In [ ]:
# ======================================================
# TEST 2: 취업확률 모델 (Train vs Test)
# ======================================================

def test_employment_overfitting(model, X_train, y_train, X_test, y_test):
    """
    취업확률 모델 오버피팅 점검
    """
    # Train
    y_train_prob = model.predict_proba(X_train)[:, 1]
    train_auc = roc_auc_score(y_train, y_train_prob)

    # Test
    y_test_prob = model.predict_proba(X_test)[:, 1]
    test_auc = roc_auc_score(y_test, y_test_prob)

    print("\n" + "="*60)
    print("🧪 [TEST] 취업확률 모델")
    print("="*60)
    print(f"Train AUC : {train_auc:.4f}")
    print(f"Test  AUC : {test_auc:.4f}")
    print(f"Gap       : {train_auc - test_auc:.4f}")

    return {
        "train_auc": train_auc,
        "test_auc": test_auc,
        "gap": train_auc - test_auc
    }


In [ ]:
test_employment_overfitting(
    employment_model,
    X_train, y_train,
    X_test, y_test
)


In [ ]:
# ======================================================
# TEST 3: Calibration Curve (정책용 필수)
# ======================================================

from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt

def test_calibration_curve(y_true, y_prob, title="Calibration Curve"):
    prob_true, prob_pred = calibration_curve(
        y_true, y_prob,
        n_bins=10,
        strategy="quantile"
    )

    plt.figure(figsize=(6, 6))
    plt.plot(prob_pred, prob_true, marker='o', label='Model')
    plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Perfect')
    plt.xlabel("예측 확률")
    plt.ylabel("실제 취업 비율")
    plt.title(title)
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()


In [ ]:
test_calibration_curve(
    y_test,
    employment_model.predict_proba(X_test)[:, 1],
    title="Employment Model Calibration (Test)"
)


In [ ]:
# ======================================================
# TEST 4: 정책 타겟팅 Lift 테스트
# ======================================================

def test_topk_lift(df, k=0.2, score_col="지원필요도점수"):
    """
    집중 타겟팅 Lift (Top-K 기준)
    """
    base_rate = df["진성쉬었음"].mean()
    n = int(len(df) * k)

    top = df.sort_values(score_col, ascending=False).head(n)
    top_rate = top["진성쉬었음"].mean()
    lift = top_rate / base_rate if base_rate > 0 else 0

    print("\n" + "="*60)
    print(f"🧪 [TEST] 집중 타겟팅 Lift (Top {int(k*100)}%)")
    print("="*60)
    print(f"대상 인원        : {n}")
    print(f"Top-K 위험비율   : {top_rate*100:.2f}%")
    print(f"전체 평균        : {base_rate*100:.2f}%")
    print(f"Lift             : {lift:.2f}배")

    return lift



In [ ]:
for k in [0.30, 0.20, 0.10]:
    test_topk_lift(df_policy, k=k)
